In [1]:
import torch
import torch.nn.functional as F
from scipy.stats import multivariate_normal
import math
import numpy as np, scipy.stats as st
import matplotlib.pyplot as plt
from matplotlib import cm
%matplotlib inline
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import random
from torch.autograd import Variable
from tqdm import tqdm

In [2]:
def sample(mu, var, nb_samples=500):
    """
    sample guassian random variable
    :param mu: torch.Tensor (features)
    :param var: torch.Tensor (features) (note: zero covariance)
    :return: torch.Tensor (nb_samples, features)
    """
    out = []
    for i in range(nb_samples):
        out += [
            torch.normal(mu, var.sqrt())
        ]
    return torch.stack(out, dim=0)

In [3]:
class Linear_net_sig(nn.Module):
    '''
    Linear binary classifier with unit init
    '''
    def __init__(self, input_dim, out_dim = 1):
        super(Linear_net_sig, self).__init__()
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(input_dim, 1)
        torch.nn.init.ones_(self.fc1.weight)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.fc1(x)
        x = self.sigmoid(x)
        return x    

In [129]:
def run_classifier_sig(net, data_x, data_y):
    '''
    trains binary classifier using SGD
    '''

    BCE = torch.nn.BCELoss()
    optimizer = optim.SGD(net.parameters(), lr=0.1, momentum=0)
    for epoch in range(100):  # loop over the dataset multiple times

        running_loss = 0.0
        # get the inputs; data is a list of [inputs, labels]
        inputs = data_x
        labels = data_y
        order = np.array(range(len(data_x)))
        np.random.shuffle(order)
        
        # in-place changing of values
        inputs[np.array(range(len(data_x)))] = inputs[order]
        labels[np.array(range(len(data_x)))] = labels[order]

        # zero the parameter gradients
        #scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, len(inputs)*100)

        # forward + backward + optimize
        outputs = net(inputs)
        #loss = -labels*torch.log2(outputs) - (1-labels)*torch.log2(1-outputs) #BCE(outputs, labels)
        #loss = torch.sum(loss)/ len(inputs)
        loss = BCE(outputs, labels) 
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        #scheduler.step()
        running_loss += loss.item()
    #print("loss " + str(loss.item()))

    #print('Finished Training')

In [5]:
def test_classifier_sig(net, data_x, data_y):
    '''
    tests binary classifier and prints accuracy
    '''
    correct = 0
    total = 0
    with torch.no_grad():
        inputs =  data_x
        labels = data_y
        outputs = net(inputs)
        predicted = torch.round(outputs.data)
        total = labels.size(0)
        for i in range(total):
            correct += predicted[i].item() == labels[i].item()
        #correct = (predicted == labels).sum()
    print('Accuracy of the network on the  test examples: %d %%' % (
        100 * correct / total))

In [126]:
class Linear_net_rej(nn.Module):
    '''
    Linear Classifier to be used for the L_CE loss
    '''
    def __init__(self, input_dim, out_dim):
        super(Linear_net_rej, self).__init__()
        # an affine operation: y = Wx + b
        self.fc = nn.Linear(input_dim, out_dim+1)
        self.fc_rej = nn.Linear(input_dim, 1)
        torch.nn.init.ones_(self.fc.weight)
        torch.nn.init.ones_(self.fc_rej.weight)
        self.softmax = nn.Softmax()

    def forward(self, x):
        out = self.fc(x)
        rej = self.fc_rej(x)
        #out = torch.cat([out,rej],1)
        out = self.softmax(out)
        return out

In [17]:
def reject_CrossEntropyLoss(outputs, m0, m1, labels, m2, n_classes):
    '''
    Implmentation of L_{CE}^{\alpha}
        outputs: network outputs
        m0: cost of deferring to expert0 cost of classifier predicting (I_{m =y})
        m1: cost of deferring to expert1 cost of classifier predicting (I_{m =y})
        labels: target
        m2:  cost of classifier predicting (alpha* I_{m\neq y} + I_{m =y})
        n_classes: number of classes
    '''
     
    batch_size = outputs.size()[0]            # batch_size
    rc = [n_classes] * batch_size
    rc = torch.tensor(rc)
    outputs =  -m0*torch.log2( outputs[range(batch_size), rc]) -m1*torch.log2( outputs[range(batch_size), rc]) - m2*torch.log2(outputs[range(batch_size), labels])   # pick the values corresponding to the labels
    return torch.sum(outputs)/batch_size



In [195]:

def run_classifier_rej(net, net_exp0, net_exp1, data_x, data_y, alpha):
    '''
    training script for L_{CE}
        net: classifier and rejector model
        net_exp1: expert1 model
        net_exp0: expert0 model
        data_x: numpy x data
        data_y: numpy y data
        alpha: hyperparam alpha for loss L_CE^{\alpha}
        '''
    optimizer = optim.SGD(net.parameters(), lr=0.1)
    #scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, len(data_x)*50)

    for epoch in range(1):  
        running_loss = 0.0
        inputs = data_x
        labels = data_y
        order = np.array(range(len(data_x)))
        np.random.shuffle(order)
        inputs[np.array(range(len(data_x)))] = inputs[order]
        labels[np.array(range(len(data_x)))] = labels[order]
        x_batches = torch.split(inputs,64)
        y_batches = torch.split(labels,64)  
        for inputs, labels in zip(x_batches, y_batches):
            optimizer.zero_grad()

            # forward + backward + optimize
            m0 = net_exp0(inputs) # need to be 0
            m1 = net_exp1(inputs)
            _, predicted0 = torch.min(m0.data, 1)
            _, predicted1 = torch.max(m1.data, 1)

            m0 = (predicted0==labels)*1
            m1 = (predicted1==labels)*1
            m2 = [0] * len(inputs)
            for j in range (0,len(inputs)):
                if (m0[j] | m1[j]):
                    m2[j] = alpha
                else:
                    m2[j] = 1
            m0 = torch.tensor(m0)
            m1 = torch.tensor(m1)
            m2 = torch.tensor(m2)
            outputs = net(inputs)
            loss = reject_CrossEntropyLoss(outputs, m0, m1, labels, m2, 2)
            #loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            #scheduler.step()
            running_loss += loss.item()
            #print("loss " + str(loss.item()))

    #print('Finished Training')



In [197]:

def test_classifier_rej(net, net_exp0, net_exp1, data_x, data_y):
    
    '''Testing script for L_{CE} loss'''
    
    correct = 0
    correct_sys = 0
    exp0 = 0
    exp0_total = 0
    exp1 = 0
    exp1_total = 0
    total = 0
    real_total = 0
    alone_correct = 0
    with torch.no_grad():
        inputs =  data_x
        labels = data_y
        m0 = net_exp0(inputs)
        m1 = net_exp1(inputs)
        _, predicted_exp0 = torch.max(m0.data, 1)
        _, predicted_exp1 = torch.max(m1.data, 1)
        outputs = net(inputs)
        _, predicted = torch.max(outputs.data, 1)
        
        '''
        if a, then defer to expert0
        if b, then defer to expert1
        if c, then no defer
        modify below
        '''

        for i in range(len(inputs)):
            r = (predicted[i] == 2).item() # if 2, then defer to expert
            if r:
                exp0 += (predicted_exp0[i] == labels[i]).item()
                correct_sys += (predicted_exp0[i] == labels[i]).item()
                exp0_total += 1
            else:
                correct += (predicted[i] == labels[i]).item() 
                correct_sys += (predicted[i] == labels[i]).item()
                total += 1
        real_total += labels.size(0)

    print("system accuracy", 100 * correct_sys / real_total)
    print("expert0 accuracy", 100 * exp0 / (exp0_total + 0.0002))
    print("expert1 accuracy", 100 * exp1 / (exp1_total + 0.0002))
    print("classifier accuracy", 100 * correct / (total + 0.0001))
    print("alone classifier", 100 * alone_correct / real_total)


    return [100 * total / real_total, 100 * correct_sys / real_total, 100 * exp0 / (exp0_total + 0.0002),
            100 * exp1 / (exp1_total + 0.0002), 100 * correct / (total + 0.0001)]

# Data Generation

In [36]:
d = 10
total_samples = 1000
#group_proportion = np.random.uniform()
group_proportion = 0.5
#if group_proportion <= 0.02:
    #group_proportion = 0.02
#if group_proportion >= 0.98:
    #group_proportion = 0.98
    #group_proportion = 0.4
cluster1_mean = torch.rand(d)*d
cluster1_var = torch.rand(d)*d
cluster1 = sample(
        cluster1_mean,
        cluster1_var,
        nb_samples= math.floor(total_samples * group_proportion * 0.5 )
    )
cluster1_labels = torch.ones([math.floor(total_samples * group_proportion * 0.5 )], dtype=torch.long)
cluster2_mean = torch.rand(d)*d
cluster2_var = torch.rand(d)*d
cluster2 = sample(
        cluster2_mean,
        cluster2_var,
        nb_samples= math.floor(total_samples * group_proportion * 0.5 )
    )
cluster2_labels = torch.zeros([math.floor(total_samples * group_proportion * 0.5 )], dtype=torch.long)
cluster3_mean = torch.rand(d)*d
cluster3_var = torch.rand(d)*d
cluster3 = sample(
        cluster3_mean,
        cluster3_var,
        nb_samples= math.floor(total_samples * (1-group_proportion) * 0.5 )
    )
cluster3_labels = torch.ones([math.floor(total_samples * (1-group_proportion) * 0.5 )], dtype=torch.long)
    
cluster4_mean = torch.rand(d)*d
cluster4_var = torch.rand(d)*d
cluster4 = sample(
        cluster4_mean,
        cluster4_var,
        nb_samples= math.floor(total_samples * (1-group_proportion) * 0.5 )
    )
cluster4_labels = torch.zeros([math.floor(total_samples * (1-group_proportion) * 0.5 )], dtype=torch.long)
    
    # test data
cluster1_test = sample(
        cluster1_mean,
        cluster1_var,
        nb_samples= math.floor(total_samples * group_proportion * 0.5 )
    )
cluster1_labels_test = torch.ones([math.floor(total_samples * group_proportion * 0.5 )], dtype=torch.long)
    
cluster2_test = sample(
        cluster2_mean,
        cluster2_var,
        nb_samples= math.floor(total_samples * group_proportion * 0.5 )
    )
cluster2_labels_test = torch.zeros([math.floor(total_samples * group_proportion * 0.5 )], dtype=torch.long)

cluster3_test = sample(
        cluster3_mean,
        cluster3_var,
        nb_samples= math.floor(total_samples * (1-group_proportion) * 0.5 )
    )
cluster3_labels_test = torch.ones([math.floor(total_samples * (1-group_proportion) * 0.5 )], dtype=torch.long)
    
cluster4_test = sample(
        cluster4_mean,
        cluster4_var,
        nb_samples= math.floor(total_samples * (1-group_proportion) * 0.5 )
    )
cluster4_labels_test = torch.zeros([math.floor(total_samples * (1-group_proportion) * 0.5 )], dtype=torch.long)
data_x_test = torch.cat([cluster1_test, cluster2_test, cluster3_test, cluster4_test])
data_y_test = torch.cat([cluster1_labels_test, cluster2_labels_test, cluster3_labels_test, cluster4_labels_test])

# Experts models

In [143]:
# expert0 model
net_exp0 = Linear_net_sig(d,1)
data_x = torch.cat([cluster4])
data_y = cluster4_labels.view(-1, 1).type(torch.float)
run_classifier_sig(net_exp0, data_x, data_y)

In [81]:
# expert1 model
net_exp1 = Linear_net_sig(d,1)
data_x = torch.cat([cluster3])
data_y = cluster3_labels.view(-1, 1).type(torch.float)
run_classifier_sig(net_exp0, data_x, data_y)

# Defer model with alpha = 0, 0.5, 1

In [48]:
experimental_data_rej1 = []
experimental_data_rej5 = []
experimental_data_rej0 = []

In [200]:
#reject
data_x = torch.cat([cluster1, cluster2, cluster3, cluster4])
data_y = torch.cat([cluster1_labels, cluster2_labels, cluster3_labels, cluster4_labels])

net_rej = Linear_net_rej(d,3)
alpha = 0
run_classifier_rej(net_rej, net_exp0, net_exp1, data_x, data_y, alpha) ####
batch_data = test_classifier_rej(net_rej, net_exp0, net_exp1, data_x_test, data_y_test) ####
experimental_data_rej0.append(batch_data)

system accuracy 98.5
expert0 accuracy 97.64239778294782
expert1 accuracy 0.0
classifier accuracy 99.38898179450473
alone classifier 0.0


/var/folders/rn/y27_vshj5d549src5ptd24d00000gq/T/ipykernel_89274/1374592031.py:41: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  m0 = torch.tensor(m0)
/var/folders/rn/y27_vshj5d549src5ptd24d00000gq/T/ipykernel_89274/1374592031.py:42: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  m1 = torch.tensor(m1)


In [198]:
net_rej = Linear_net_rej(d,3)
alpha = 0.5
run_classifier_rej(net_rej, net_exp0, net_exp1, data_x, data_y, alpha) ####
batch_data = test_classifier_rej(net_rej, net_exp0, net_exp1, data_x_test, data_y_test) ####
experimental_data_rej5.append(batch_data)

system accuracy 50.0
expert0 accuracy 0.0
expert1 accuracy 0.0
classifier accuracy 49.9999950000005
alone classifier 0.0


/var/folders/rn/y27_vshj5d549src5ptd24d00000gq/T/ipykernel_89274/1374592031.py:41: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  m0 = torch.tensor(m0)
/var/folders/rn/y27_vshj5d549src5ptd24d00000gq/T/ipykernel_89274/1374592031.py:42: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  m1 = torch.tensor(m1)


In [199]:
net_rej = Linear_net_rej(d,3)
alpha = 1
run_classifier_rej(net_rej, net_exp0, net_exp1, data_x, data_y, alpha) ####
batch_data = test_classifier_rej(net_rej, net_exp0, net_exp1, data_x_test, data_y_test) ####
experimental_data_rej1.append(batch_data)

system accuracy 50.0
expert0 accuracy 0.0
expert1 accuracy 0.0
classifier accuracy 49.9999950000005
alone classifier 0.0


/var/folders/rn/y27_vshj5d549src5ptd24d00000gq/T/ipykernel_89274/1374592031.py:41: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  m0 = torch.tensor(m0)
/var/folders/rn/y27_vshj5d549src5ptd24d00000gq/T/ipykernel_89274/1374592031.py:42: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  m1 = torch.tensor(m1)
